In [114]:
import requests
import os
import glob
from natsort import natsorted
import json
import csv
from datetime import datetime
import pandas as pd
import socket

Test Script

In [115]:
payload = {'statement': 'select 1;', 'pretty': 'true', 'client_context_id': 'xyz'}
response = requests.get('http://localhost:19002/query/service', params = payload)
response.json()
# row['metrics']['executionTime'] = float(row['metrics']['executionTime'].replace('ms', ''))
# row['metrics']['executionTime']

{'requestID': 'ce810f46-f93e-4bc4-85e8-0a22df93be07',
 'clientContextID': 'xyz',
 'signature': {'*': '*'},
 'results': [{'$1': 1}],
 'plans': {},
 'status': 'success',
 'metrics': {'elapsedTime': '108.352762ms',
  'executionTime': '102.224543ms',
  'resultCount': 1,
  'resultSize': 15,
  'processedObjects': 0}}

Ask user for benchmark type (tiny or big)

In [116]:
benchmarkType = input("Enter tiny for tiny benchmark or large for large benchmark: ")

if benchmarkType == "tiny":
    isTiny = True
elif benchmarkType == "large":
    isTiny = False
else:
    print("Input is invalid. Defaulting to tiny benchmark.")
    benchmarkType = "tiny"
    isTiny = True

if isTiny:
    dataverse = "USE TinyBenchmark;"
else:
    dataverse = "USE BigBenchmark;"

dataverse += "\n"
print(dataverse)

Enter tiny for tiny benchmark or large for large benchmark: tiny
USE TinyBenchmark;



Check the current working directory

In [117]:
cwd = os.getcwd()
cwd += "/"
cwd

'/Users/andretran/Documents/Projects/AsterixDB/'

Get the path of dataset

In [118]:
addr = input("Enter address: ")

# print(ip)

datasetsPath = ''

if addr == '127.0.0.1':
    datasetsPath = cwd + "datasets/"
elif addr == '1':
    datasetsPath = "/local_data/downloads/datasets/"
else:
    print("Address is not recognized. Defaulting to 127.0.0.1.")
    addr = '127.0.0.1'
    datasetsPath = cwd + "datasets/"

print(addr)
print(datasetsPath)

Enter address: 127.0.0.1
127.0.0.1
/Users/andretran/Documents/Projects/AsterixDB/datasets/


Get the queries based on if the benchmark is tiny or big

In [119]:
# path = '/Users/andretran/Documents/Projects/AsterixDB/'
sqlExt = "*.sql"
queriesPath = cwd + "queries/" + sqlExt
print(queriesPath)
tinyQueriesPath = cwd + "tinyQueries/" + sqlExt
bigQueriesPath = cwd + "bigQueries/" + sqlExt

queries = glob.glob(queriesPath)
tinyQueries = glob.glob(tinyQueriesPath)
bigQueries = glob.glob(bigQueriesPath)

if isTiny:
    benchmarkQueries = queries + tinyQueries
else:
    benchmarkQueries = queries + bigQueries

benchmarkQueries = natsorted(benchmarkQueries, key=os.path.basename)
print(benchmarkQueries)

/Users/andretran/Documents/Projects/AsterixDB/queries/*.sql
['/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/1a_Create_All.sql', '/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/2a_Load_All.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/3_Point-Count_Buildings.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/4_Point-Count_RoadNetwork.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/5_Point-Count_AllNodes.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/6_Equals_Polygon-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/7a_Equals_Point-Point.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/8_Disjoint_Polygon-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/9_Intersects_Line-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/queries/10_Intersects_Point-Polygon.sql', '/Users/andretran/Documents/Projects/AsterixDB/tinyQueries/11a_Intersects_Point-Line.sql',

Create a results folder to store benchmark results

In [120]:
resultsPath = cwd + 'results/'

isExists = os.path.exists(resultsPath)
print(isExists)

if not isExists:
    os.makedirs(path)
    print("results directory created")
print(resultsPath)

True
/Users/andretran/Documents/Projects/AsterixDB/results/


Create json and csv files which will hold results

In [122]:
# Import Module

# Folder Path
# path = "/Users/andretran/Documents/Projects/AsterixDB/queries"

# all_files = glob.glob('/Users/andretran/Documents/Projects/AsterixDB/queries/*.sql')

# Change the directory
# os.chdir(resultsPath)
# if os.path.exists(cwd + 'results/'):

dateString = datetime.utcnow().strftime('%Y-%m-%d %H_%M_%S.%f')[:-3]
jsonFile = resultsPath + benchmarkType + "_" + dateString + ".json"
csvFile = resultsPath + benchmarkType + "_" + dateString + ".csv"

def create_json_file():
    if os.path.exists(jsonFile):
        os.remove(jsonFile)
        print('JSON file deleted')
    with open(jsonFile, mode='w', encoding='utf-8') as f:
        json.dump([], f)
        print('JSON file created')
        
def create_csv_file():
    if os.path.exists(csvFile):
        os.remove(csvFile)
        print('CSV file deleted')
    with open(csvFile, mode='w', encoding='utf-8') as f:
        header = ['requestID', 'clientContextID', 'status', 'elapsedTime', 'resultCount', 'resultSize', 'processedObjects', 'timestamp']
        writer = csv.writer(f)
        # write the header
        writer.writerow(header)
        print('CSV file created')
        
create_json_file()
create_csv_file()

JSON file created
CSV file created


Run the benchmark and save results to csv and json files

In [123]:
def benchmark_queries(file_path, index):
    with open(file_path, 'r') as f:
        statement = f.read()
        if index != 0:
            statement = dataverse + statement
        if index == 22 or 1:
            statement = statement.replace('$PATH$', addr + '://' + datasetsPath)
            
        print(statement)
        payload = {'statement': statement, 'pretty': 'true', 'client_context_id': os.path.basename(file_path), 'readonly': False}
        timestamp = datetime.now()
        response = requests.post('http://localhost:19002/query/service', params = payload)
        row = response.json()
        print(row['status'])
        with open(jsonFile, "r") as of:
            data = json.load(of)
        data.append(row)
        with open(jsonFile, "w") as of:
            json.dump(data, of)
        with open(csvFile, mode='a', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([row['requestID'], row['clientContextID'], row['status'], row['metrics']['elapsedTime'], row['metrics']['resultCount'], row['metrics']['resultSize'], row['metrics']['processedObjects'], timestamp])


# iterate through all file
for index, file_path in enumerate(benchmarkQueries):
    # Check whether file is in text format or not
    print(os.path.basename(file_path))
    benchmark_queries(file_path, index)

print("done")

1a_Create_All.sql
DROP DATAVERSE TinyBenchmark IF EXISTS;
CREATE DATAVERSE TinyBenchmark;
USE TinyBenchmark;

CREATE TYPE RoadNetworkType AS {
	id: UUID,
	g: geometry,
	attr0: string?,
	attr1: string?,
	attr2: string?,
	attr3: string?,
	attr4: string?
};

CREATE TYPE BuildingsType AS {
	id: UUID,
	g: geometry,
	attr0: string?,
	attr1: string?
};

CREATE TYPE AllNodesType AS {
	id: UUID,
	g: geometry,
	attr0: string?,
	attr1: string?
};

CREATE DATASET RoadNetwork (RoadNetworkType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET Buildings (BuildingsType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET AllNodes (AllNodesType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET TempAllNodes (AllNodesType) PRIMARY KEY id AUTOGENERATED;
CREATE DATASET TempRoadNetwork (RoadNetworkType) PRIMARY KEY id AUTOGENERATED;
success
2a_Load_All.sql
USE TinyBenchmark;
-- USE TinyBenchmark;

-- 'path'='127.0.0.1:///Users/andretran/Documents/Projects/AsterixDB/datasets/OSM2015_road-network.json'

LOAD DATASET RoadNet

37_Land-Information-Management_Average-SqFt-Buildings-States.sql
USE TinyBenchmark;
SELECT States.name, AVG(st_area(Buildings.g)) as AvgArea
FROM States, Buildings
WHERE st_contains(States.g, Buildings.g)
GROUP BY States.name
success
done


- uniqueID
- clientCopntextID as filename
- check for status if failed or success
- elapsed time
- timestamp

- create sample output file (json and csv)

- collect memory usage over time (timeseries)
    - peak memory (how tight a process is in terms of memory comsumption)